# Data Preparation

Load and clean the session-level data from Excel.

In [1]:
# 📦 Import required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline


# Load Excel file
file_path = 'SOF Data.xlsx'
df = pd.read_excel(file_path)
df.head()

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

## Loading and Cleaning Data

In [3]:
df_read = pd.read_excel(file_path)
df_read.head(10) # Display the first few rows to verify

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)
df.drop(columns=['Package Title'], inplace=True)
df = df[df['Transaction Status'] == 'Paid']
df.drop(columns=['Transaction Status'], inplace=True)
df = df.rename(columns={'Sum of Session Price': 'Price Paid'})
df = df.reset_index(drop=True)

In [ ]:
# Make a copy of the DataFrame to avoid modifying the original
df_plot = df.copy()

# Convert Session Start Time strings to datetime.time objects
df_plot['Session Start Time'] = pd.to_datetime(df_plot['Session Start Time'], format='%H:%M:%S').dt.time

# Get sorted order of times (unique and sorted)
time_order = sorted(df_plot['Session Start Time'].unique())


# Get sorted order of times
time_order = sorted(df_plot['Session Start Time'].dropna().unique())
plt.figure(figsize=(10, 5))
sns.countplot(data=df_plot, x='Session Type')
plt.title('Countplot of Session Type')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Define target session types
target_types = ['Play', 'Round Robin', 'Clinic', 'Lesson', 'Team', 'Tournament', 'League', 'Unavailable']

# Filter the dataframe
df_filtered = df[df['Session Type'].isin(target_types)]

# Calculate percentages
session_counts = df_filtered['Session Type'].value_counts(normalize=True).reindex(target_types).fillna(0) * 100

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x=session_counts.index, y=session_counts.values, palette='Set2')
plt.title('Percentage of Selected Session Types')
plt.xlabel('Session Type')
plt.ylabel('Percentage (%)')
plt.xticks(rotation=45, ha='right')

# Add percentage labels above bars
for i, pct in enumerate(session_counts.values):
    plt.text(i, pct + 0.5, f'{pct:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 5))
sns.countplot(data=df_plot, x='Day of Week')
plt.title('Countplot of Day of Week')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
plt.figure(figsize=(14, 5))
ax = sns.countplot(data=df_plot, x='Session Start Time', order=time_order)
plt.title('Countplot of Session Start Time')
ax.set_xticklabels([t.strftime('%H:%M') for t in time_order], rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Sessions by Day of Week
sns.countplot(data=df, x='Day of Week', order=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
plt.title('Session Count by Day of Week')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Sessions by Session Start Time
df['Hour'] = pd.to_datetime(df['Session Start Time'], format='%H:%M:%S', errors='coerce').dt.hour
sns.countplot(data=df, x='Hour')
plt.title('Session Count by Hour of Day')
plt.tight_layout()
plt.show()

In [ ]:

DF_play = DF_play.reset_index(drop=True)
# Step 1: Rename the column
DF_play = DF_play.rename(columns={'Sum of Session Price': 'Price Paid'})

In [ ]:


# Aggregate revenue and sessions per user
user_stats = DF_play.groupby('Full Name').agg({
    'Price Paid': 'sum',
    'Full Name': 'count'
}).rename(columns={'Full Name': 'Session Count'})

# Grid search across possible cutoff combinations
results = []

# Loop through lower and upper cutoffs
for low in range(1, 21):        # Lower cutoff for Medium
    for high in range(low + 1, 101):  # Upper cutoff for Heavy
        # Assign groups
        def classify(count):
            if count >= high:
                return 'Heavy'
            elif count >= low:
                return 'Medium'
            else:
                return 'Light'

        user_stats['Group'] = user_stats['Session Count'].apply(classify)

        # Aggregate revenue by group
        group_summary = user_stats.groupby('Group').agg(
            Total_Revenue=('Price Paid', 'sum'),
            Num_Users=('Price Paid', 'count')
        )
        group_summary['Avg_Revenue_per_User'] = group_summary['Total_Revenue'] / group_summary['Num_Users']
        group_summary['Cutoff_Low'] = low
        group_summary['Cutoff_High'] = high

        # Add to results if all groups are represented
        if group_summary.shape[0] == 3:
            group_summary['Total_Revenue_All'] = group_summary['Total_Revenue'].sum()
            group_summary['Revenue_Share'] = group_summary['Total_Revenue'] / group_summary['Total_Revenue_All']
            group_summary['Cutoffs'] = f"{low}-{high}"
            results.append(group_summary.reset_index())

# Combine results
all_results = pd.concat(results)

# Focus on summary stats for each cutoff combo
summary = all_results.pivot_table(
    index='Cutoffs',
    columns='Group',
    values='Revenue_Share'
)

# Sort by how much revenue Heavy users contribute
top_cutoffs = summary.sort_values(by='Heavy', ascending=False).head(200)
print(top_cutoffs)